## Get points of interest

In [ ]:
# Imports
import osmnx as ox
import folium

In [ ]:
# Get all food-related amenities
place = "Berlin, Germany"
berlin_coord = [52.520008, 13.404954]
tags = {"amenity": ["restaurant", "cafe", "fast_food", "bar", "pub", "ice_cream", "biergarten", 
                    "food_court", "canteen"]} # if all items of a tag call dictionary entry as "tag name": True
poi = ox.geometries_from_place(place, tags=tags)
poi_points = poi[poi.geom_type == "Point"]
poi.head()

In [ ]:
# If we want, we can also plot them in osmnx
# Note though: plotting as .html takes forever. So I'd suggest a different method or within geopandas altogether

# m = folium.Map(berlin_coord, zoom_start=10)
# locs = zip(poi_points.geometry.y, poi_points.geometry.x)
# for id, location in enumerate(locs):
#     print(id)
#     folium.CircleMarker(location=location).add_to(m)
#     m.save("poi.html")

## Visualize some stuff

In [ ]:
# Imports
import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
# Load datasets
prefix = r"C:\Users\Michl\Documents\GitHub\data-ambassadors-parking-project\data\datathon"
lor_gdf = gpd.read_file(prefix+"/lor_friedrichshain_kreuzberg.geojson")
estimated_parking_spots_kfz_gdf = gpd.read_file(prefix+"/parking/estimated_parking_spots_kfz.geojson")

In [ ]:
# Some general plotting (code from Sebastian)
def get_lor_map():
    return lor_gdf.explore(
        column="PLR_NAME",
        legend=True,
        legend_kwds={"colorbar": False},
        style_kwds={"opacity": 0.4, "fillOpacity": 0.4},
        name="Friedrichshain-Kreuzberg"
    )
def plot_geodataframe(gdf: gpd.GeoDataFrame):
    ax = gdf.plot(figsize=(20, 20), alpha=0.5, edgecolor="k")
    cx.add_basemap(ax, crs=gdf.crs)

In [ ]:
get_lor_map()

## Confine POI map to KB-FH

In [ ]:
# Plotting with lots of parameters seems to tank the memory availability, hence drop unnecessary (debatable) information
poi_points_s = poi_points.dropna(axis=1)
poi_points_s["name"] = poi_points["name"]
# Project to the same coordinate system
poi_points_s = poi_points_s.to_crs("EPSG:25833")

In [ ]:
# We look for the intersection of all pois and our Bezirk
poi_lor = gpd.sjoin(poi_points_s, lor_gdf, predicate='intersects')

In [ ]:
poi_lor.explore(m=lor_gdf.explore())

In [ ]:
# This is the parking spots as estimated from OSM
estimated_parking_spots_kfz_gdf.explore()

In [ ]:
# Define a circular region around each poi (currently 20m)
# At this stage we could also define any type of shape
# We might need to think about overlaps here...
poi_lor_buffered = poi_lor.copy()
poi_lor_buffered["geometry"] = poi_lor["geometry"].buffer(20)
poi_lor_buffered = poi_lor_buffered.drop(labels=["index_right"], axis=1)
# Show the area together with the parking spots
poi_lor_buffered.explore(m=estimated_parking_spots_kfz_gdf.explore())

## Calculate parking spots per POI

In [ ]:
# Intersect parking spots and area
intersect_parking = gpd.sjoin(poi_lor_buffered, estimated_parking_spots_kfz_gdf)
# Calculate parking spots
intersect_parking["capacity"] = intersect_parking["capacity"].apply(int)
dfpivot = pd.pivot_table(intersect_parking, index="name_left", columns="capacity", aggfunc={"capacity":len})
dfpivot.columns = dfpivot.columns.droplevel()
dfpivot.index.names = ["name"]
parking_counts = dfpivot.sum(axis=1)
parking_counts = parking_counts.to_frame("No. parking spots")

In [ ]:
# Merge back with the main dataframe
poi_lor_counts = poi_lor.merge(parking_counts.reset_index(), on="name", how="left")
poi_lor_counts["No. parking spots"] = poi_lor_counts["No. parking spots"].fillna(0)

In [ ]:
poi_lor_counts